In [11]:
import pandas as pd
import io
import boto3
import openpyxl
import time
from openpyxl.styles import Border, Side
from tempfile import NamedTemporaryFile

#os.chdir("/users/morgana.sartor/desktop/development/github/claro_free_report_poc")

class ClaroFreeAutomateReports:

    def __init__(self):
        self.source_database_name = 'claro_free'
        self.source_table_name = 'rel_campaign'
        self.s3_bucket = 'prezao-free-report-test'
        self.s3_output_querys = f's3://{self.s3_bucket}/source-querys'
        self.s3_target_url = f's3://{self.s3_bucket}/reports'
        self.s3_model_url = f's3://{self.s3_bucket}/model'
        self.region_name = 'us-east-1'

        self.athena_client = boto3.client('athena', region_name=self.region_name)
        self.s3_resource = boto3.resource('s3', region_name=self.region_name)
        self.s3_client = boto3.client('s3', region_name=self.region_name)

        # Valores de células do excel
        self.report_infos_overview_column = 3
        self.report_infos_start_line = 6
        
        self.report_infos_consolidaded_column = 13
        self.report_infos_consolidaded_start_line = 6
        
        self.data_table_start_column = 2
        self.data_table_last_column = 7
        self.data_table_start_line = 6
        self.data_table_total_line = 5

        self.limit_index = 28
        
        self.thin = Side(border_style="thin", color="000000")

    def get_filename_from_athena_query(self, query):
        filename = ''
        try:
            response = self.athena_client.start_query_execution(
                QueryString = query,
                    QueryExecutionContext={
                    'Database': self.source_database_name
                    },
                    ResultConfiguration={
                    'OutputLocation': self.s3_output_querys,
                    }
            )
            filename = response['QueryExecutionId']
            print('Execution ID: ' + response['QueryExecutionId'])

            query_status = None
            while query_status == 'QUEUED' or query_status == 'RUNNING' or query_status is None:
                query_status = self.athena_client.get_query_execution(QueryExecutionId=response["QueryExecutionId"])['QueryExecution']['Status']['State']
                print(f'Query status: {query_status}')
                print('...')
                if query_status == 'FAILED' or query_status == 'CANCELLED':
                    raise Exception('Athena query with the string "{}" failed or was cancelled'.format(self.query))
                time.sleep(10)
            print(f'Query {query} finished.')
        except Exception as e:
            print(e)

        return filename

    def get_df_from_athena_query(self, query):
        filename = self.get_filename_from_athena_query(query)
        result = ''

        response = self.s3_resource \
                    .Bucket(self.s3_bucket) \
                    .Object(key='source-querys/' + filename + '.csv') \
                    .get()

        result = pd.read_csv(io.BytesIO(response['Body'].read()), encoding='utf8') 

        return result

    def all_reports_on_table(self):
        query = 'select distinct name as report from claro_free.rel_campaign'
        self.all_reports_on_table = self.get_df_from_athena_query(query)

    def all_reports_on_s3(self):
        s3_bucket = self.s3_resource.Bucket(self.s3_bucket)
        folder = 'reports/'
        self.reports_already_created = [
            f.key.split(folder + "/")[1] 
            for f in s3_bucket.objects.filter(Prefix=folder).all()
            if len(f.key.split(folder + "/")) > 1
        ]

    def get_reports_to_be_created(self):
        reports_to_create = list(set(self.all_reports_on_table['report']) - set(self.reports_already_created))
        return reports_to_create

    def get_reports_to_be_increased(self):
        reports_to_increase = list(set(self.all_reports_on_table['report']).intersection(self.reports_already_created))
        return reports_to_increase

    def create_report(self, campaign_name, wb):
        print(f'Report: {campaign_name}')
        
        query = f"select * from claro_free.rel_campaign where name = '{campaign_name}'"
        results = self.get_df_from_athena_query(query)
        
        print(f'Data of {campaign_name}: \n{results}')
        
        results.sort_values(by=['date'], ascending=True, inplace=True)

        # Inserir dados na primeira planilha
        sheet_ranges = wb['Dashboard']

        #advertiser = report_data.advertiser.unique()
        #sheet_ranges.cell(row=report_infos_start_line, column=report_infos_overview_column).value = advertiser[0]    
        campaign = campaign_name
        sheet_ranges.cell(row=self.report_infos_start_line+1, column=self.report_infos_overview_column).value = campaign[0]
        volume = results.volume.unique()
        sheet_ranges.cell(row=self.report_infos_start_line+2, column=self.report_infos_overview_column).value = volume[0] 
        value = results.cpm.unique()
        sheet_ranges.cell(row=self.report_infos_start_line+3, column=self.report_infos_overview_column).value = value[0]
        date_data = results['date'].reset_index(drop=True)
        sheet_ranges.cell(row=self.report_infos_start_line+6, column=self.report_infos_overview_column).value = date_data[0]
    
        size_data = len(date_data)
        finish_date = date_data[size_data - 1]
    
        impression = results['impression'].reset_index(drop=True)
        clicked = results['clicked'].reset_index(drop=True)
        complete = results['complete'].reset_index(drop=True)

        # Inserir dados na segunda planilha
        sheet_ranges = wb['Dados']

        last_index = self.data_table_start_line + len(date_data)

        # Limpa todas as células do modelo
        for cell in range(self.data_table_start_line, self.limit_index):
            sheet_ranges.cell(row=cell, column=2).value = ''

        # Retira formato da última célula
        for col in range(self.data_table_start_column, self.data_table_last_column):
            sheet_ranges.cell(row=self.limit_index, column=col).border = None

        # Preenche dados
        for cell in range(self.data_table_start_line, last_index):
        
            sheet_ranges.cell(row=cell, column=self.data_table_start_column).value = date_data[cell - self.data_table_start_line]
            sheet_ranges.cell(row=cell, column=self.data_table_start_column+1).value = impression[cell - self.data_table_start_line]
            sheet_ranges.cell(row=cell, column=self.data_table_start_column+2).value = clicked[cell - self.data_table_start_line]
            sheet_ranges.cell(row=cell, column=self.data_table_start_column+3).value = f'=D{cell}/C{cell}'
            sheet_ranges.cell(row=cell, column=self.data_table_start_column+4).value = complete[cell - self.data_table_start_line]
            sheet_ranges.cell(row=cell, column=self.data_table_start_column+5).value = f'=F{cell}/D{cell}'
    
            # Mantem a formatação
            sheet_ranges.cell(row=cell, column=self.data_table_start_column).border = Border(left=self.thin)
            sheet_ranges.cell(row=cell, column=self.data_table_last_column).border = Border(right=self.thin)
        
        sheet_ranges.cell(row=self.data_table_total_line, column=self.data_table_start_column+1).value = f'=SUM(C{self.data_table_start_line}:C{last_index - 1})'
        sheet_ranges.cell(row=self.data_table_total_line, column=self.data_table_start_column+2).value = f'=SUM(D{self.data_table_start_line}:D{last_index - 1})'
        sheet_ranges.cell(row=self.data_table_total_line, column=self.data_table_start_column+3).value = f'=D{last_index - 1}/C{last_index - 1}'
        sheet_ranges.cell(row=self.data_table_total_line, column=self.data_table_start_column+4).value = f'=SUM(F{self.data_table_start_line}:F{last_index - 1})'
        sheet_ranges.cell(row=self.data_table_total_line, column=self.data_table_start_column+5).value = f'=F{last_index - 1}/D{last_index - 1}'

        # Adiciona formatação na última linha
        for col in range(self.data_table_start_column, self.data_table_last_column+1):
            if col == self.data_table_start_column:
                sheet_ranges.cell(row=cell, column=col).border = Border(left=self.thin, bottom=self.thin)
            elif col == self.data_table_last_column:
                sheet_ranges.cell(row=cell, column=col).border = Border(right=self.thin, bottom=self.thin)
            else:
                sheet_ranges.cell(row=cell, column=col).border = Border(bottom=self.thin)
    
        sheet_ranges = wb['Dashboard']
        sheet_ranges.cell(row=self.report_infos_consolidaded_start_line, column=self.report_infos_consolidaded_column).value = f'=Dados!C{self.data_table_total_line}'
        sheet_ranges.cell(row=self.report_infos_consolidaded_start_line+1, column=self.report_infos_consolidaded_column).value = f'=Dados!D{self.data_table_total_line}'
        sheet_ranges.cell(row=self.report_infos_consolidaded_start_line+2, column=self.report_infos_consolidaded_column).value = f'=Dados!F{self.data_table_total_line}'
        sheet_ranges.cell(row=self.report_infos_consolidaded_start_line+3, column=self.report_infos_consolidaded_column).value = f'=Dados!G{self.data_table_total_line}'

        folder_name = campaign_name.replace('/', '.').replace(' ','-')
        file_name = f'reports/{folder_name}/{folder_name}({finish_date}).xlsx'
        self.s3_client.put_object(Bucket=self.s3_bucket, Key=file_name)
        with NamedTemporaryFile() as tmp:
            temp_file = f'tmp/tmp.xlsx'
            wb.save(temp_file)
            self.s3_resource.Bucket(self.s3_bucket).upload_file(Filename=temp_file, Key=file_name)

        print(f'Created report {report} in s3://{self.s3_bucket}/{file_name}\n')

claro_free_automate_reports = ClaroFreeAutomateReports()
# claro_free_automate_reports.all_reports_on_table()
response = claro_free_automate_reports.s3_resource.Bucket(claro_free_automate_reports.s3_bucket).Object(key='source-querys/dedc2018-f65b-4de0-aba4-10fbd745b5a4.csv').get()
claro_free_automate_reports.all_reports_on_table = pd.read_csv(io.BytesIO(response['Body'].read()), encoding='utf8') 
claro_free_automate_reports.all_reports_on_s3()

print(f'Reports on table: {claro_free_automate_reports.all_reports_on_table}\n')
print(f'Reports already created: {claro_free_automate_reports.reports_already_created}\n')

reports_to_be_created = claro_free_automate_reports.get_reports_to_be_created()
reports_to_be_increased = claro_free_automate_reports.get_reports_to_be_increased()
    
print(f'Reports to create: {reports_to_be_created}\n')
print(f'Reports to increase: {reports_to_be_increased}\n')

model_object = claro_free_automate_reports.s3_resource.Bucket(claro_free_automate_reports.s3_bucket).Object(key='model').get()
model_path = io.BytesIO(model_object['Body'].read())

for report in reports_to_be_created:
    workbook = openpyxl.load_workbook(model_path)
    claro_free_automate_reports.create_report(report, workbook)
    workbook.close()

Reports on table:                                            report
0       Campanha Calhau - Fallback Programática 2
1                                 Dobro de moedas
2   Campanha Calhau - Fallback (Claro Promo) - V2
3                   Desafio da Fortuna - Mediação
4                          Mídia Claro free 10/02
5                              Fevereiro em Dobro
6                          Desafio Rivalo - 27.06
7                  Campanha Rivelo Calhau - 27.06
8                    #rumo ao diamante - Mediação
9                               #rumo ao diamante
10                              Avulso - Mediação
11                     Dobro de moedas - Mediação
12                             Carnaval de Moedas
13                             Desafio da fortuna
14                                         Avulso

Reports already created: []

Reports to create: ['Dobro de moedas - Mediação', 'Campanha Rivelo Calhau - 27.06', 'Dobro de moedas', '#rumo ao diamante - Mediação', 'Fevereiro em Dob

Report: Dobro de moedas
Execution ID: 07b98754-3fc0-4dd7-b032-3ffeffa92495
Query status: QUEUED
...
Query status: RUNNING
...
Query status: RUNNING
...
Query status: RUNNING
...
Query status: SUCCEEDED
...
Query select * from claro_free.rel_campaign where name = 'Dobro de moedas' finished.
Data of Dobro de moedas: 
           date                          campaignUuid             name  \
0    2021-04-02  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
1    2021-03-28  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
2    2021-05-22  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
3    2021-06-14  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
4    2021-05-27  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
..          ...                                   ...              ...   
121  2021-03-22  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
122  2021-03-11  d90b2203-558e-4ad9-97cb-2ea74ec79d3c  Dobro de moedas   
123  2021-04-24  

Data of Fevereiro em Dobro: 
          date                          campaignUuid                name  \
0   2021-02-26  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
1   2021-02-25  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
2   2021-02-20  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
3   2021-02-21  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
4   2021-02-17  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
5   2021-02-28  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
6   2021-02-22  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
7   2021-02-13  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
8   2021-02-11  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
9   2021-03-03  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
10  2021-02-12  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fevereiro em Dobro   
11  2021-03-04  20dea3c9-90b2-4ece-ae9a-4312498d7e92  Fever

Created report Mídia Claro free 10/02 in s3://prezao-free-report-test/reports/Mídia-Claro-free-10.02/Mídia-Claro-free-10.02(2021-07-07).xlsx

Report: Campanha Calhau - Fallback Programática 2
Execution ID: ffa70e77-6a09-4bce-93e6-a5eef2b60eef
Query status: QUEUED
...
Query status: SUCCEEDED
...
Query select * from claro_free.rel_campaign where name = 'Campanha Calhau - Fallback Programática 2' finished.
Data of Campanha Calhau - Fallback Programática 2: 
          date                          campaignUuid  \
0   2021-03-07  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
1   2021-03-06  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
2   2021-03-05  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
3   2021-03-14  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
4   2021-03-23  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
5   2021-02-26  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
6   2021-03-10  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
7   2021-03-13  a1ef41cf-860d-4164-971d-4bfcd45dce2a   
8   2021-03-03  a1ef41cf-860d-4164-97

Created report #rumo ao diamante in s3://prezao-free-report-test/reports/#rumo-ao-diamante/#rumo-ao-diamante(2021-07-07).xlsx

Report: Campanha Calhau - Fallback (Claro Promo) - V2
Execution ID: e653c0ec-d3ae-4322-8027-8f2b295e4cb2
Query status: QUEUED
...
Query status: RUNNING
...
Query status: RUNNING
...
Query status: RUNNING
...
Query status: RUNNING
...
Query status: SUCCEEDED
...
Query select * from claro_free.rel_campaign where name = 'Campanha Calhau - Fallback (Claro Promo) - V2' finished.
Data of Campanha Calhau - Fallback (Claro Promo) - V2: 
          date                          campaignUuid  \
0   2021-06-17  3f794e6b-b385-4ef9-b9c7-06dbb290814e   
1   2021-07-05  3f794e6b-b385-4ef9-b9c7-06dbb290814e   
2   2021-07-01  3f794e6b-b385-4ef9-b9c7-06dbb290814e   
3   2021-06-05  3f794e6b-b385-4ef9-b9c7-06dbb290814e   
4   2021-05-29  3f794e6b-b385-4ef9-b9c7-06dbb290814e   
5   2021-07-06  3f794e6b-b385-4ef9-b9c7-06dbb290814e   
6   2021-06-15  3f794e6b-b385-4ef9-b9c7-06dbb290

Created report Campanha Calhau - Fallback (Claro Promo) - V2 in s3://prezao-free-report-test/reports/Campanha-Calhau---Fallback-(Claro-Promo)---V2/Campanha-Calhau---Fallback-(Claro-Promo)---V2(2021-07-07).xlsx

Report: Carnaval de Moedas
Execution ID: 0ad893f6-79fa-48d9-86ca-c2af70bcdd54
Query status: QUEUED
...
Query status: SUCCEEDED
...
Query select * from claro_free.rel_campaign where name = 'Carnaval de Moedas' finished.
Data of Carnaval de Moedas: 
          date                          campaignUuid                name  \
0   2021-02-27  a65d8c07-cf48-48d4-824b-e25ec222f95d  Carnaval de Moedas   
1   2021-03-04  a65d8c07-cf48-48d4-824b-e25ec222f95d  Carnaval de Moedas   
2   2021-03-01  a65d8c07-cf48-48d4-824b-e25ec222f95d  Carnaval de Moedas   
3   2021-02-19  a65d8c07-cf48-48d4-824b-e25ec222f95d  Carnaval de Moedas   
4   2021-03-02  a65d8c07-cf48-48d4-824b-e25ec222f95d  Carnaval de Moedas   
5   2021-02-23  a65d8c07-cf48-48d4-824b-e25ec222f95d  Carnaval de Moedas   
6   2021-

Created report Avulso - Mediação in s3://prezao-free-report-test/reports/Avulso---Mediação/Avulso---Mediação(2021-07-07).xlsx

Report: Desafio da Fortuna - Mediação
Execution ID: 75f8f476-ebd8-43fc-b06e-0230588abac3
Query status: QUEUED
...
Query status: SUCCEEDED
...
Query select * from claro_free.rel_campaign where name = 'Desafio da Fortuna - Mediação' finished.
Data of Desafio da Fortuna - Mediação: 
          date                          campaignUuid  \
0   2021-07-02  e391fb35-2e68-4393-8d29-52d83e299832   
1   2021-06-28  e391fb35-2e68-4393-8d29-52d83e299832   
2   2021-06-30  e391fb35-2e68-4393-8d29-52d83e299832   
3   2021-06-21  e391fb35-2e68-4393-8d29-52d83e299832   
4   2021-06-24  e391fb35-2e68-4393-8d29-52d83e299832   
5   2021-07-06  e391fb35-2e68-4393-8d29-52d83e299832   
6   2021-06-23  e391fb35-2e68-4393-8d29-52d83e299832   
7   2021-06-22  e391fb35-2e68-4393-8d29-52d83e299832   
8   2021-06-18  e391fb35-2e68-4393-8d29-52d83e299832   
9   2021-07-04  e391fb35-2e68-43

Query select * from claro_free.rel_campaign where name = 'Desafio Rivalo - 27.06' finished.
Data of Desafio Rivalo - 27.06: 
         date                          campaignUuid                    name  \
0  2021-06-27  7457915c-d5f3-477a-86ed-374191b8c12f  Desafio Rivalo - 27.06   
1  2021-06-26  7457915c-d5f3-477a-86ed-374191b8c12f  Desafio Rivalo - 27.06   
2  2021-06-26  7457915c-d5f3-477a-86ed-374191b8c12f  Desafio Rivalo - 27.06   
3  2021-06-27  7457915c-d5f3-477a-86ed-374191b8c12f  Desafio Rivalo - 27.06   
4  2021-06-25  7457915c-d5f3-477a-86ed-374191b8c12f  Desafio Rivalo - 27.06   
5  2021-06-25  7457915c-d5f3-477a-86ed-374191b8c12f  Desafio Rivalo - 27.06   

  media_type campaign_type zone_type  delivered  impression  converted  \
0       vast           CPV   Desafio     242087      318448     103380   
1       vast           CPV   Desafio      48388       61829      19236   
2       vast           CPV   Desafio      48388       61829      19236   
3       vast           CP